###  Import Statements


In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from scipy.stats import skew

### Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

# Load the Data

The first column in the .csv file just has the row numbers, so it will be used as the index.

In [3]:
data = pd.read_csv('boston.csv', index_col=0)

### Understand the Boston House Price Dataset

---------------------------

**Characteristics:**  

    :Number of Instances: 506

    :Number of Attributes: 13 numeric/categorical predictive. The Median Value (attribute 14) is the target.

    :Attribute Information (in order):
        1. CRIM     per capita crime rate by town
        2. ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        3. INDUS    proportion of non-retail business acres per town
        4. CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        5. NOX      nitric oxides concentration (parts per 10 million)
        6. RM       average number of rooms per dwelling
        7. AGE      proportion of owner-occupied units built prior to 1940
        8. DIS      weighted distances to five Boston employment centres
        9. RAD      index of accessibility to radial highways
        10. TAX      full-value property-tax rate per $10,000
        11. PTRATIO  pupil-teacher ratio by town
        12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        13. LSTAT    % lower status of the population
        14. PRICE     Median value of owner-occupied homes in $1000's
        
    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of [UCI ML housing dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/). This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University. You can find the [original research paper here](https://deepblue.lib.umich.edu/bitstream/handle/2027.42/22636/0000186.pdf?sequence=1&isAllowed=y).


# Preliminary Data Exploration 🔎

In [ ]:
# Check the shape of the data
print(f'Shape of data: {data.shape}')

# Check column names
print(f'Column names: {data.columns}')

# Display the first few rows of the data
print("First few rows of the data:")
print(data.head())

# Display the last few rows of the data
print("Last few rows of the data:")
print(data.tail())

# Check the number of rows in each column
print("Number of rows in each column:")
print(data.count())

# Check for missing values and duplicates
print("Data information:")
print(data.info())
print(f'Any NaN values? {data.isna().values.any()}')
print(f'Number of duplicates: {data.duplicated().sum()}')

# Check the statistical summary of the numerical columns
print("Statistical summary of numerical columns:")
print(data.describe())

# Visualize missing data
import missingno as msno
print("Nullity matrix:")
msno.matrix(data)

# Check data types
print("Data types:")
print(data.dtypes)

# Check unique values for categorical variables
for col in data.columns:
    if data[col].dtype == 'object':
        print(f'{col}: {data[col].unique()}')

## Descriptive Statistics

In [ ]:
# Descriptive statistics
print(data.describe())

# Calculate average students per teacher
average_students_per_teacher = data['PTRATIO'].mean()
print(f"Average students per teacher: {average_students_per_teacher:.2f}")

# Calculate average home price
average_price = data['PRICE'].mean()
print(f"Average home price: {average_price:.2f}")

# Minimum and maximum values of 'CHAS'
min_chas = data['CHAS'].min()
max_chas = data['CHAS'].max()
print(f"Minimum CHAS value: {min_chas}")
print(f"Maximum CHAS value: {max_chas}")

# Count the number of tracts that bound the Charles River and those that don't
print(data['CHAS'].value_counts())

# Minimum and maximum number of rooms per dwelling
min_rooms = data['RM'].min()
max_rooms = data['RM'].max()
print(f"Minimum number of rooms per dwelling: {min_rooms:.2f}")
print(f"Maximum number of rooms per dwelling: {max_rooms:.2f}")

## Visualise the Features

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 5))

# Home Prices
sns.displot(data['PRICE'], bins=50, kde=True, color='#2196f3')
plt.title(f'1970s Home Values in Boston. Average: ${(1000*data.PRICE.mean()):.6}')
plt.xlabel('Price in 000s')
plt.ylabel('Nr. of Homes')
plt.show()

# Distance to Employment
sns.displot(data.DIS, bins=50, kde=True, color='#2196f3')
plt.title(f'Distance to Employment Centres. Average: {(data.DIS.mean()):.2}')
plt.xlabel('Weighted Distance to 5 Boston Employment Centres')
plt.ylabel('Nr. of Homes')
plt.show()

# Number of Rooms
sns.displot(data.RM, kde=True, color='#2196f3')
plt.title(f'Distribution of Rooms in Boston. Average: {data.RM.mean():.2}')
plt.xlabel('Average Number of Rooms')
plt.ylabel('Nr. of Homes')
plt.show()

# Access to Highways
sns.displot(data['RAD'], bins=24, kde=True, color='#2196f3')
plt.xlabel('Accessibility to Highways')
plt.ylabel('Nr. of Houses')
plt.show()

# Next to the River
river_access = data['CHAS'].value_counts()
bar = px.bar(x=['No', 'Yes'], y=river_access.values, color=river_access.values, color_continuous_scale=px.colors.sequential.haline, title='Next to Charles River?')
bar.update_layout(xaxis_title='Property Located Next to the River?', yaxis_title='Number of Homes', coloraxis_showscale=False)
bar.show()

# Understand the Relationships in the Data

### Run a Pair Plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(10, 10))

# Pair Plot
sns.pairplot(data, corner=True)
plt.tight_layout()
plt.show()

# Distance from Employment vs. Pollution
with sns.axes_style('darkgrid'):
    sns.jointplot(x=data['DIS'], y=data['NOX'], height=8, kind='scatter', color='deeppink', joint_kws={'alpha': 0.5})
    plt.title('Distance from Employment vs. Pollution')
    plt.xlabel('Distance from Employment')
    plt.ylabel('Nitric Oxide Pollution')
    plt.tight_layout()
    plt.show()

# Proportion of Non-Retail Industry vs. Pollution
with sns.axes_style('darkgrid'):
    sns.jointplot(x=data['INDUS'], y=data['NOX'], height=7, color='darkgreen', joint_kws={'alpha': 0.5})
    plt.title('Proportion of Non-Retail Industry vs. Pollution')
    plt.xlabel('Proportion of Non-Retail Industry')
    plt.ylabel('Nitric Oxide Pollution')
    plt.tight_layout()
    plt.show()

# % of Lower Income Population vs Average Number of Rooms
with sns.axes_style('darkgrid'):
    sns.jointplot(x=data['LSTAT'], y=data['RM'], height=7, color='orange', joint_kws={'alpha': 0.5})
    plt.title('% of Lower Income Population vs Average Number of Rooms')
    plt.xlabel('% of Lower Income Population')
    plt.ylabel('Average Number of Rooms')
    plt.tight_layout()
    plt.show()

# % of Lower Income Population vs Home Price
with sns.axes_style('darkgrid'):
    sns.jointplot(x=data['LSTAT'], y=data['PRICE'], height=7, color='crimson', joint_kws={'alpha': 0.5})
    plt.title('% of Lower Income Population vs Home Price')
    plt.xlabel('% of Lower Income Population')
    plt.ylabel('Home Price')
    plt.tight_layout()
    plt.show()

# Number of Rooms vs Home Value
with sns.axes_style('darkgrid'):
    sns.jointplot(x=data['RM'], y=data['PRICE'], height=7, color='darkblue', joint_kws={'alpha': 0.5})
    plt.title('Number of Rooms vs Home Value')
    plt.xlabel('Number of Rooms')
    plt.ylabel('Home Value')
    plt.tight_layout()
    plt.show()

# Split Training & Test Dataset

We *can't* use all 506 entries in our dataset to train our model. The reason is that we want to evaluate our model on data that it hasn't seen yet (i.e., out-of-sample data). That way we can get a better idea of its performance in the real world.

In [ ]:
# Split Training & Test Dataset
features = data.drop('PRICE', axis=1)
target = data['PRICE']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=10)

# Print the percentage of training and test data
train_pct = 100 * len(x_train) / len(features)
test_pct = 100 * len(x_test) / len(features)

print(f'Training data: {train_pct:.2f}%')
print(f'Test data: {test_pct:.2f}%')

# Multivariable Regression

In a previous lesson, we had a linear model with only a single feature (our movie budgets). This time we have a total of 13 features. Therefore, our Linear Regression model will have the following form:

$$ PR \hat ICE = \theta _0 + \theta _1 RM + \theta _2 NOX + \theta _3 DIS + \theta _4 CHAS ... + \theta _{13} LSTAT$$

In [ ]:
# Run the regression on the training dataset
regr = LinearRegression()
regr.fit(x_train, y_train)
rsquared = regr.score(x_train, y_train)

print(f'Training data r-squared: {round(rsquared, 2)}')

# Evaluate the coefficients of the model
regr_coef = pd.DataFrame(data=regr.coef_, index=x_train.columns, columns=['Coefficient']).round(2)
regr_coef.index.name = 'Feature'
print(regr_coef)

# Calculate the premium for having an extra room
premium = regr_coef.loc['RM'].values[0] * 1000
print(f'The price premium for having an extra room is ${premium:.2f}')

### Analyse the Estimated Values & Regression Residuals

The next step is to evaluate our regression. How good our regression is depends not only on the r-squared. It also depends on the **residuals** - the difference between the model's predictions ($\hat y_i$) and the true values ($y_i$) inside `y_train`.

```
predicted_values = regr.predict(X_train)
residuals = (y_train - predicted_values)
```



In [ ]:
# Calculate predicted values and residuals
predicted_vals = regr.predict(x_train)
residuals = (y_train - predicted_vals)

# Original Regression of Actual vs. Predicted Prices
plt.figure(dpi=100, figsize=(8, 6))
plt.scatter(x=y_train, y=predicted_vals, c='indigo', alpha=0.6)
plt.plot(y_train, y_train, color='cyan')
plt.title('Actual vs Predicted Prices: $y_i$ vs $\hat y_i$', fontsize=17)
plt.xlabel('Actual prices 000s $y_i$', fontsize=14)
plt.ylabel('Predicted prices 000s $\hat y_i$', fontsize=14)
plt.show()

# Residuals vs Predicted values
plt.figure(dpi=100, figsize=(8, 6))
plt.scatter(x=predicted_vals, y=residuals, c='indigo', alpha=0.6)
plt.title('Residuals vs Predicted Values', fontsize=17)
plt.xlabel('Predicted Prices $\hat y_i$', fontsize=14)
plt.ylabel('Residuals', fontsize=14)
plt.show()

# Calculate the mean and skewness of the residuals
resid_mean = round(residuals.mean(), 2)
resid_skew = round(residuals.skew(), 2)

# Residual Distribution Chart
sns.displot(residuals, kde=True, color='indigo')
plt.title(f'Residuals Skew ({resid_skew}) Mean ({resid_mean})')
plt.show()

### Data Transformations for a Better Fit

We have two options at this point:

1. Change our model entirely. Perhaps a linear model is not appropriate.
2. Transform our data to make it fit better with our linear model.

Let's try a data transformation approach.


In [ ]:
# Visualize the original price distribution
tgt_skew = data['PRICE'].skew()
sns.displot(data['PRICE'], kde='kde', color='green')
plt.title(f'Normal Prices. Skew is {tgt_skew:.3}')
plt.show()

# Perform log transformation on the price data
y_log = np.log(data['PRICE'])

# Visualize the log price distribution
sns.displot(y_log, kde=True)
plt.title(f'Log Prices. Skew is {y_log.skew():.3}')
plt.show()

# Compare the skewness of the original and log price distributions
# The distribution with a skew closer to zero is a better candidate for the linear model
if abs(y_log.skew()) < abs(tgt_skew):
    better_distribution = 'Log Prices'
else:
    better_distribution = 'Normal Prices'

print(f"The distribution with a skew closer to zero is: {better_distribution}")

# Plot the mapping of original prices to log prices
plt.figure(dpi=150)
plt.scatter(data.PRICE, np.log(data.PRICE))
plt.title('Mapping the Original Price to a Log Price')
plt.xlabel('Actual $ Price in thousands')
plt.ylabel('Log Price')
plt.show()


## Regression using Log Prices

Using log prices instead, our model has changed to:

$$ \log (PR \hat ICE) = \theta _0 + \theta _1 RM + \theta _2 NOX + \theta_3 DIS + \theta _4 CHAS + ... + \theta _{13} LSTAT $$


In [ ]:
# Split the data into training and test sets using the log prices
new_target = np.log(data['PRICE'])  # Use log prices
features = data.drop('PRICE', axis=1)

X_train, X_test, log_y_train, log_y_test = train_test_split(features, new_target, test_size=0.2, random_state=10)

# Perform linear regression using log prices
log_regr = LinearRegression()
log_regr.fit(X_train, log_y_train)

# Calculate the r-squared of the regression on the training data
log_rsquared = log_regr.score(X_train, log_y_train)

# Make predictions using the log price regression
log_predictions = log_regr.predict(X_train)
log_residuals = (log_y_train - log_predictions)

# Print the r-squared of the regression on the training data
print(f'Training data r-squared: {log_rsquared:.2f}')

# Determine if the fit of the model has improved compared to before based on the r-squared value
if log_rsquared > rsquared:
    improvement = 'improved'
else:
    improvement = 'not improved'

print(f'The fit of the model has {improvement} compared to before based on the r-squared value.')


## Evaluating Coefficients with Log Prices

In [ ]:
# Create a DataFrame to store the coefficients of the new regression model
df_coef = pd.DataFrame(data=log_regr.coef_, index=X_train.columns, columns=['coef'])

# Print out the coefficients of the new regression model
print(df_coef)

## Regression with Log Prices & Residual Plots

In [ ]:
# Scatter plot of Actual vs. Predicted Log Prices
plt.scatter(x=log_y_train, y=log_predictions, c='navy', alpha=0.6)
plt.plot(log_y_train, log_y_train, color='cyan')
plt.title(f'Actual vs Predicted Log Prices: $y_i$ vs $\hat y_i$ (R-Squared {log_rsquared:.2f})', fontsize=17)
plt.xlabel('Actual Log Prices $y_i$', fontsize=14)
plt.ylabel('Predicted Log Prices $\hat y_i$', fontsize=14)
plt.show()

# Scatter plot of Original Actual vs. Predicted Prices
plt.scatter(x=y_train, y=predicted_vals, c='indigo', alpha=0.6)
plt.plot(y_train, y_train, color='cyan')
plt.title(f'Original Actual vs Predicted Prices: $y_i$ vs $\hat y_i$ (R-Squared {rsquared:.3f})', fontsize=17)
plt.xlabel('Actual prices 000s $y_i$', fontsize=14)
plt.ylabel('Predicted prices 000s $\hat y_i$', fontsize=14)
plt.show()

# Scatter plot of Residuals vs. Predicted values (Log prices)
plt.scatter(x=log_predictions, y=log_residuals, c='navy', alpha=0.6)
plt.title('Residuals vs Fitted Values for Log Prices', fontsize=17)
plt.xlabel('Predicted Log Prices $\hat y_i$', fontsize=14)
plt.ylabel('Residuals', fontsize=14)
plt.show()

# Scatter plot of Residuals vs. Predicted values
plt.scatter(x=predicted_vals, y=residuals, c='indigo', alpha=0.6)
plt.title('Original Residuals vs Fitted Values', fontsize=17)
plt.xlabel('Predicted Prices $\hat y_i$', fontsize=14)
plt.ylabel('Residuals', fontsize=14)
plt.show()

# Calculate the mean and skew for the residuals using log prices
log_resid_mean = round(log_residuals.mean(), 2)
log_resid_skew = round(log_residuals.skew(), 2)

# Plot the distribution of residuals using log prices
sns.displot(log_residuals, kde=True, color='navy')
plt.title(f'Log price model: Residuals Skew ({log_resid_skew}) Mean ({log_resid_mean})')
plt.show()

# Plot the distribution of residuals in the original model
sns.displot(residuals, kde=True, color='indigo')
plt.title(f'Original model: Residuals Skew ({resid_skew}) Mean ({resid_mean})')
plt.show()

# Compare Out of Sample Performance

The *real* test is how our model performs on data that it has not "seen" yet. This is where our `X_test` comes in.

In [20]:
# Calculate and print the r-squared for the original model on the test dataset
original_model_r2 = regr.score(X_test, y_test)
print(f'Original Model Test Data r-squared: {original_model_r2:.2f}')

# Calculate and print the r-squared for the log model on the test dataset
log_model_r2 = log_regr.score(X_test, log_y_test)
print(f'Log Model Test Data r-squared: {log_model_r2:.2f}')

Original Model Test Data r-squared: 0.67
Log Model Test Data r-squared: 0.74


# Predict a Property's Value using the Regression Coefficients

Our preferred model now has an equation that looks like this:

$$ \log (PR \hat ICE) = \theta _0 + \theta _1 RM + \theta _2 NOX + \theta_3 DIS + \theta _4 CHAS + ... + \theta _{13} LSTAT $$

The average property has the mean value for all its charactistics:

In [22]:
# Calculate the average values of the features
average_vals = features.mean().values

# Create a DataFrame with the average values as a single row
property_stats = pd.DataFrame(data=average_vals.reshape(1, len(features.columns)), columns=features.columns)

# Predict the value of the average property using the stats above
log_estimate = log_regr.predict(property_stats)[0]
print(f'The log price estimate is ${log_estimate:.3f}')

# Convert Log Prices to Actual Dollar Values
dollar_est = np.exp(log_estimate) * 1000
print(f'The property is estimated to be worth ${dollar_est:.2f}')

# Define Property Characteristics
next_to_river = True
nr_rooms = 8
students_per_classroom = 20 
distance_to_town = 5
pollution = data.NOX.quantile(q=0.75)  # high
amount_of_poverty =  data.LSTAT.quantile(q=0.25)  # low

# Set Property Characteristics in the property_stats DataFrame
property_stats['RM'] = nr_rooms
property_stats['PTRATIO'] = students_per_classroom
property_stats['DIS'] = distance_to_town

if next_to_river:
    property_stats['CHAS'] = 1
else:
    property_stats['CHAS'] = 0

property_stats['NOX'] = pollution
property_stats['LSTAT'] = amount_of_poverty

# Make prediction for the property with specified characteristics
log_estimate = log_regr.predict(property_stats)[0]
print(f'The log price estimate is ${log_estimate:.3f}')

# Convert Log Prices to Actual Dollar Values
dollar_est = np.exp(log_estimate) * 1000
print(f'The property is estimated to be worth ${dollar_est:.2f}')


The log price estimate is $3.030
The property is estimated to be worth $20703.18
The log price estimate is $3.250
The property is estimated to be worth $25792.03
